# PyData Carolinas VM Assignment Bot

0. Listen for "vm please" requests to @vmbot.
1. Get a list of all VMs tagged pydata2016 from the SoftLayer (SL) API
2. Filter out all VMs that already have a non-blank note indicating the "owner" of the VM
3. Randomly pick one unassigned VM
4. Fetch the notebook and dashboard server credentials stored in the VM user metadata
5. Store the Slack user ID in the VM notes field
6. Respond with links to the services on the VM and credentials
7. Show recent log messages and VM ownership in simple dashboard tables

In [ ]:
# !pip install git+https://github.com/parente/dizzybot -U

In [ ]:
import json
import os
import random
import SoftLayer
import collections
from datetime import datetime
from urth.widgets.widget_channels import channel
from dizzybot import Dizzybot
from tornado import ioloop

If True, don't save the owner info back to SoftLayer to avoid exhausting the pool. Otherwise, write the owner info to the SoftLayer VM notes field.

In [ ]:
DEV = True

Make sure `SOFTLAYER_USER`, `SOFTLAYER_API_KEY`, and `SLACK_TOKEN` are set in the environment. Or set them here.

In [ ]:
SOFTLAYER_API_KEY = os.getenv('SOFTLAYER_API_KEY')
SOFTLAYER_USER = os.getenv('SOFTLAYER_USER')
SLACK_TOKEN = os.getenv('SLACK_TOKEN')
TRIGGER = 'vm please'
HEALTH_CHECK_INTERVAL_MS = 10000

Globals used across functions and inspected for admin dashboard display.

In [ ]:
def notify_vm_status():
    '''
    Put the FQDN and its owner on the vms channel as status.
    '''
    channel('vms').set('status', 
                       [(instance['fullyQualifiedDomainName'], instance.get('notes', '')) for instance in instances])

### SoftLayer Client

In [ ]:
sl_client = SoftLayer.create_client_from_env(username=SOFTLAYER_USER, api_key=SOFTLAYER_API_KEY)

In [ ]:
vm_mgr = SoftLayer.VSManager(sl_client)

In [ ]:
instances = vm_mgr.list_instances(tags=['pydata2016'], mask='id,notes,fullyQualifiedDomainName,userData')
notify_vm_status()

In [ ]:
def assign_vm(username):
    '''
    Assign a virtual machine to the user. If the user already owns a VM,
    return that same VM instead of assigning a new one. 
    
    Raises IndexError if there are no unassigned VMs left to dole out.
    '''
    available = []
    for instance in instances:
        owner = instance.get('notes', '').strip()
        if owner == username:
            instance['config'] = json.loads(instance['userData'][0]['value'])
            return instance
        elif not owner and instance.get('userData', []):
            available.append(instance)
    
    instance = random.choice(available)
    if not DEV:
        # Assign VMs for real if we're not in dev mode
        # Otherwise, only mark them locally
        rv = vm_mgr.edit(instance['id'], notes=username)
    # Update the notes locally too so we don't have to hit the API again
    instance['notes'] = username
    # Make the user data more easily accessible
    instance['config'] = json.loads(instance['userData'][0]['value'])
    notify_vm_status()
    return instance

### Slack Client

In [ ]:
class VMBot(Dizzybot):
    def log(self, text):
        '''Override logging to track datetime too.'''
        super(VMBot, self).log((str(datetime.now()), text))
        channel('log').set('recent', reversed(list(self.recent)))
    
    def on_event(self, evt):
        '''
        Assign a VM to the user that asked for one and respond with
        the URL and credentials the user needs to access the notebook
        server and dashboard server running on it.
        '''
        if evt.get('type') != 'message' or 'reply_to' in evt:
            return
        
        if TRIGGER not in evt.get('text', '').lower():
            return
            
        # Get username from message
        try:
            user_id = evt['user']
        except KeyError:
            return self.log('abort: could not get user from message')

        # Assign a VM to the user
        try:
            vm = assign_vm(user_id)
        except IndexError as e:
            self.log('abort, out of VMs: {}'.format(e))
            text = 'Oh no! We have no VMs left. Tell an instructor!'
        else:
            text = '''Hi, <@{user_id}>. Here's your machine details:
    Jupyter Notebook: http://{fullyQualifiedDomainName}:8888 (password: {config[nb_password]})
    Jupyter Dashboards: http://{fullyQualifiedDomainName}:3000 (username: pydata, password: {config[db_password]})
    '''.format(user_id=user_id, **vm)
        
        # Respond to the requester
        self.respond(evt, text)

In [ ]:
try:
    bot.stop()
except NameError:
    pass
bot = VMBot(SLACK_TOKEN)

In [ ]:
bot.start()

### Admin UI

Shows recent messages and current VM assignment status.

In [ ]:
import declarativewidgets as dw 
dw.init()

In [ ]:
%%html
<template is="urth-core-bind" channel="log">
    <table>
        <caption>Recent messages</caption>
        <thead>
            <tr>
                <th></th>
                <th>Local time</th>
                <th>Type</th>
                <th>Text</th>
            </tr>
        </thead>
        <tbody>
        <template is="dom-repeat" items="[[recent]]">
            <tr>
                <td>[[index]]</td>
                <td>[[item.0]]</td>
                <td>[[item.1.type]]</td>
                <td>[[item.1.text]]</td>
            </tr>
        </template>
        </tbody>
</template>

In [ ]:
%%html
<template is="urth-core-bind" channel="vms">
    <table>
        <caption>VM assignments</caption>
        <thead>
            <tr>
                <th></th>
                <th>Host</th>
                <th>Owner</th>
            </tr>
        </thead>
        <tbody>
        <template is="dom-repeat" items="[[status]]">
            <tr>
                <td>[[index]]</td>
                <td>[[item.0]]</td>
                <td>[[item.1]]</td>
            </tr>
        </template>
        </tbody>
</template>